In [1]:
import matplotlib.pyplot as plt
import os
import numpy as np
from PIL import Image
import codecs
Image.MAX_IMAGE_PIXELS = None
from tqdm.notebook import tqdm

In [2]:
### Load Province information
prov = {}
file = open("input files/abw_provinces.txt","r")
contents = file.read().splitlines()
file.close()
for line in range(0, len(contents)-1, 15):
    ids = int(contents[line].split("=")[0].split('\t')[-1])
    prov[ids] = {
        'loc': [None]*2,
        'terrain': contents[line+1].split('"')[1],
        'culture': contents[line+2].split('"')[1],
        'religion': contents[line+3].split('"')[1],
        'trade_goods': contents[line+4].split('"')[1],
        'civilization_value': int(contents[line+5].split('=')[1]),
        'barbarian_power': int(contents[line+6].split('=')[1]),
        'province_rank': contents[line+7].split('"')[1],
        'pops': contents[line+8],
        'holysite': contents[line+9],
        'port_building': contents[line+10],
        'minor_pops1': contents[line+11],
        'minor_pops2': contents[line+12]
    }

###Load Province centers
file = open("input files/combat_locators.txt","r")
contents = file.read().splitlines()
file.close()
for line in range(8, len(contents)-4, 6):
    ids = int(contents[line].split("=")[1])
    if ids==0: continue
    temp = contents[line+1].split(" ")[1:-1]
    prov[ids]['loc'][0] = int(float(temp[0]))
    prov[ids]['loc'][1] = 6144-int(float(temp[2]))

### Load port locators to see which province can have one
file = open("input files/port_locators.txt","r")
contents = file.read().splitlines()
file.close()
coastalprovs = []
for line in range(8, len(contents)-4, 6):
    ids = int(contents[line].split("=")[1])
    coastalprovs.append(ids)

### Load countries from setup
file = open("input files/setup_countriesonly.txt","r")
contents = file.read().splitlines()
file.close()
country = {}
for line in range(2, len(contents)-4, 9):
    tag = contents[line].split(' ')[0].split('\t')[2]
    temp = contents[line+1].split('#')[1].split(',')
    country[tag] = {
        'rgb': tuple([ int(x) for x in temp ]),
        'government': contents[line+2],
        'culture': contents[line+3].split('=')[1].split(' ')[1],
        'religion': contents[line+4].split('=')[1].split(' ')[1],
        'regnal': contents[line+5],
        'capital': contents[line+6],
        'owned': []
    }

### Load areas
file = open("input files/areas.txt","r")
contents = file.read().splitlines()
file.close()
areas = {}
temp = len(contents)
for line in range(0, len(contents)-3, 4):
    name = contents[line].split(' =')[0]
    areas[name] = list(map(int,contents[line+1].split('= { ')[1].split(' }')[0].split(' ')))

### Load regions
file = open("input files/regions.txt","r")
contents = file.read().splitlines()
file.close()
regions = {}
temp = len(contents)
for line in range(0, len(contents)-3, 4):
    name = contents[line].split(' =')[0]
    regions[name] = contents[line+1].split('= { ')[1].split(' }')[0].split(' ')

### Load cultures (and save a reference file)
culture = {}
cultpath = 'input files/cultures/'
for filename in os.listdir(cultpath):
    with open(cultpath+filename, 'r') as file:
        contents = file.read().splitlines()
    # Get culture group's color
    rgb = contents[1].split('{ ')[1].split(' }')[0].split(' ')
    rgb = [ int(x) for x in rgb ]
    mincolor = np.argmin(rgb)
    
    # Find subculture header and end
    start, finish = 0, 0
    for line, value in enumerate(contents): 
        if '\tculture =' in value: start = line+1
        if start!=0:
            if '\t}'==value: 
                finish = line
                break

    # Get subculture names, save with unique color
    for i in range(start,finish):
        value = contents[i][2:]
        if value[0]=='\t' or value[0]=='}': continue
        rgb[mincolor] += 1
        culture[value.split(' =')[0]] = tuple(rgb[:])

#Check no culture has repeated colors
for name in culture:
    for name2 in culture:
        if name==name2: continue
        elif culture[name]==culture[name2]: raise ValueError(f'Attention, {name} and {name2} share the same rgb')

with open('output files/culturesReference.txt', 'w') as file:
    for name in culture:
        file.write(f'{name} : {culture[name]}\n\n')
    file.write(f'A total of {len(culture)} cultures found')

In [3]:
### CULTURE
map_country = Image.open(r'input files/maps/cultures.png').convert('RGB')
for ids in prov:
    if prov[ids]['trade_goods'] == '': continue
    pos = (prov[ids]['loc'][0], prov[ids]['loc'][1])
    pixelcolor = map_country.getpixel(pos)

    #Iterate through every culture 
    for name in culture:
        if culture[name]==pixelcolor:
            prov[ids]['culture'] = name
            break

In [4]:
### COUNTRY and buildings
buildings = {}
map_country = Image.open(r'input files/maps/countries.png').convert('RGB')
for ids in range(1,len(prov)+1):
    #Check if it's habitable, get province coordinates and color
    if prov[ids]['trade_goods'] == '':
        if prov[ids]['terrain'] != 'ocean' and prov[ids]['terrain'] != 'coastal_terrain':
            prov[ids]['culture'] = 'primitiveculture'
            prov[ids]['religion'] = 'animist_rel'
        continue
    pos = (prov[ids]['loc'][0], prov[ids]['loc'][1])
    pixelcolor = map_country.getpixel(pos)
    
    #Ignore if it's not colored
    if 3 == sum([1 for a, b in zip(pixelcolor, (255,255,255)) if a == b]): continue

    #Iterate through the countries checking if it belongs to one of them
    for tag in country:
        if 3 == sum([1 for a, b in zip(pixelcolor, country[tag]['rgb']) if a == b]):
            #Adding province to country's ownership
            country[tag]['owned'].append(ids)
            
            #Check if it's a city, in which we put a fortress
            if prov[ids]['province_rank']=='city':
                buildings[ids] = ['fort1']
                #Check if it's a coastal city, in which we place a port
                if ids in coastalprovs:
                    prov[ids]['port_building'] = '\t\tport_building=1'
            break

### Write the country & building setup
with open("input files/00_default.txt","r") as file:
    contents = file.read().splitlines()

with codecs.open('output files/00_default.txt','w', 'utf-8') as file:
    file.write('\ufeff')
    for line in contents: file.write(line+'\n')

    file.write('\nprovinces = {\n')
    for ids in buildings:
        file.write(f'\t{ids} = {"{"}\n')
        if 'fort1' in buildings[ids]: file.write('\t\tfortress_building = 1\n')
        file.write('\t}\n')
    file.write('}')
    
    file.write('\n')
    file.write('country = {\n\tcountries = {')
    for tag in country:
        file.write('\n\t\t'+tag+' = {')
        file.write('\n\t\t\t#'+str(country[tag]['rgb'][0])+','+str(country[tag]['rgb'][1])+','+str(country[tag]['rgb'][2]))
        file.write('\n'+country[tag]['government'])
        file.write('\n\t\t\tprimary_culture = '+country[tag]['culture'])
        file.write('\n\t\t\treligion = '+country[tag]['religion'])
        file.write('\n'+country[tag]['regnal']+'\n'+country[tag]['capital'])
        file.write('\n\t\t\town_control_core = 	{ ')
        for ids in country[tag]['owned']: file.write(str(ids)+' ')
        file.write('}\n\t\t}')
    file.write('\n\t}\n}')
    

In [5]:
### RELIGION
f = open("input files/00_religions.txt","r")
contents = f.read().splitlines()
f.close()

religion = {}
for line in range(1,len(contents)):
    if '#' not in contents[line]: continue
    name = contents[line].split(' ')[0]
    temp = contents[line+1].split('{ ')[1].split(' }')[0].split(' ')
    religion[name] = {
        'rgb': tuple([ int(x) for x in temp ])
    }

map_country = Image.open(r'input files/maps/religions.png').convert('RGB')
for ids in range(1,len(prov)+1):
    #Check if it's habitable, get province coordinates and color
    if prov[ids]['trade_goods'] == '': continue
    pos = (prov[ids]['loc'][0], prov[ids]['loc'][1])
    pixelcolor = map_country.getpixel(pos)

    #Iterate through the religions
    for name in religion:
        if 3 == sum([1 for a, b in zip(pixelcolor, religion[name]['rgb']) if a == b]):
            
            #Setting province religion to the country's
            prov[ids]['religion'] = name
            break

In [6]:
### CULTURE AND RELIGION DIFFUSION
nsearches = 5
for ids in tqdm(prov):
    if prov[ids]['trade_goods'] == '': continue
    pos = [prov[ids]['loc'][0], prov[ids]['loc'][1]]

    npops = int(prov[ids]['pops'].split('Total-')[1])

    search = []
    index = []
    for ids2 in prov:
        if prov[ids2]['trade_goods'] == '': continue

        search.append((prov[ids2]['loc'][0]-pos[0])**2 +(prov[ids2]['loc'][1]-pos[1])**2)
        index.append(ids2)
    
    best = np.argsort(search)[:nsearches]

    relculList = []
    for ids2 in [index[i] for i in best]:
        relculList.append(prov[ids2]['religion']+'+'+prov[ids2]['culture'])

    names, counts = np.unique(relculList, return_counts=True)
    sortindex = np.argsort(-counts)

    #Majority
    index = sortindex[0]
    prov[ids]['religion'], prov[ids]['culture'] = names[index].split('+')
    count = max(int(round(npops*counts[index]/nsearches)),1)
    prov[ids]['pops'] = f'\t\tfreemen={"{"}amount= {count} {"}"} # Total-{npops}'

    #First minority
    if len(names)>=2:
        index = sortindex[1]
        count = int(round(npops*counts[index]/nsearches))
        if count==0: prov[ids]['minor_pops1'] = '\t\t#'
        else: 
            temp = names[index].split('+')
            prov[ids]['minor_pops1'] = f'\t\tfreemen={"{"}amount= {count} culture="{temp[1]}" religion="{temp[0]}" {"}"}'

    else: prov[ids]['minor_pops1'] = '\t\t#'

    if len(names)>=3:
        index = sortindex[2]
        count = int(round(npops*counts[index]/nsearches))
        if count==0: prov[ids]['minor_pops2'] = '\t\t#'
        else:
            temp = names[index].split('+')
            prov[ids]['minor_pops2'] = f'\t\tfreemen={"{"}amount= {count} culture="{temp[1]}" religion="{temp[0]}" {"}"}'

    else: prov[ids]['minor_pops2'] = '\t\t#'

  0%|          | 0/5692 [00:00<?, ?it/s]

In [7]:
### TERRAIN
f = open("input files/00_terrains.txt","r")
contents = f.read().splitlines()
f.close()

terrains = {}
for line in range(1,len(contents)):
    if contents[line]!='#': continue
    name = contents[line+1].split(' =')[0]
    temp = contents[line+2].split('{ ')[1].split(' }')[0].split(' ')
    
    terrains[name] = {
        'rgb': tuple([ int(float(x)*255) for x in temp ])
    }

map_country = Image.open(r'input files/maps/terrain.png').convert('RGB')
for ids in range(1,len(prov)+1):
    #Check if it's habitable, get province coordinates and color
    if prov[ids]['terrain'] == 'impassable_terrain': continue
    pos = (prov[ids]['loc'][0], prov[ids]['loc'][1])
    pixelcolor = map_country.getpixel(pos)

    #Iterate through the terrains
    for name in terrains:
        if 3 == sum([1 for a, b in zip(pixelcolor, terrains[name]['rgb']) if a == b]):
            #Setting province terrain to the country's
            prov[ids]['terrain'] = name
            continue #should be break

In [8]:
### Base population ONE TIME assignment
popdensity = {}
popdensity['very_low'] = { 
    'rgb': tuple([ 255, 0, 0 ]),
    'density-range': [1, 1] }
popdensity['low'] = { 
    'rgb': tuple([ 255, 0, 191 ]),
    'density-range': [2, 5] }
popdensity['medium'] = { 
    'rgb': tuple([ 255, 119, 0 ]),
    'density-range': [6, 11] }
popdensity['high'] = { 
    'rgb': tuple([ 255, 242, 0 ]),
    'density-range': [12, 18] }
popdensity['very_high'] = { 
    'rgb': tuple([ 50, 255, 0 ]),
    'density-range': [19, 30] }

map_country = Image.open(r'input files/maps/population.png').convert('RGB')
totalpops = 0
for ids in range(1,len(prov)+1):
    #Check if its supposed to have population
    if prov[ids]['pops'][-1] == '#': continue

    #Get the location and extract the color of the pop density
    pos = (prov[ids]['loc'][0], prov[ids]['loc'][1])
    pixelcolor = map_country.getpixel(pos)

    #Iterate through the densities
    for density in popdensity:
        if 3 == sum([1 for a, b in zip(pixelcolor, popdensity[density]['rgb']) if a == b]):
            #Setting province religion to the country's
            drange = popdensity[density]['density-range']
            numberofpops = np.random.randint(drange[0],drange[1]+1)
            prov[ids]['pops'] = '\t\tfreemen={amount= '+str(numberofpops)+' }'
            break

In [9]:
### Trade good ONE TIME assignment
# List of available trade goods
tradegoods = ['grain','salt','iron','horses','wine','wood','amber','stone','fish','spices','elephants','papyrus','cloth','wild_game','precious_metals','steppe_horses','cattle','earthware','dye','fur','olive','leather','base_metals','woad','marble','honey','incense','hemp','vegetables','gems','camel','glass','silk','dates']
# Frequency for each trade good
freq = [100,20,1,5,10,30,5,4,20,2,2,1,10,4,1,2,70,10,1,4,5,10,6,2,6,10,1,10,60,1,1,4,1,8]
temp = [tradegoods[0]]*int(freq[0])
for i in range(1,len(tradegoods)):
    temp += [tradegoods[i]]*int(freq[i])

# Assignment
for ids in prov:
    #Check if it's habitable, get province coordinates and color
    if prov[ids]['culture'] == '': continue
    prov[ids]['trade_goods'] = np.random.choice(temp)

In [6]:
### City rank ONE TIME assignment
for area in areas:
    temp = []
    for ids in areas[area]:
        temp.append(int(prov[ids]['pops'].split('amount= ')[1].split(' }')[0]))
        prov[ids]['province_rank'] = 'settlement'
    index = areas[area][np.argmax(temp)]
    prov[index]['province_rank'] = 'city'

In [8]:
def write_provinces(filename, prov):
    with codecs.open('output files/'+filename+'.txt', 'w', 'utf-8') as file:
        file.write('\ufeff')
        for ids in prov:
            file.write('\t'+str(ids)+'={\n')
            file.write('\t\tterrain="'+prov[ids]['terrain']+'"\n')
            file.write('\t\tculture="'+prov[ids]['culture']+'"\n')
            file.write('\t\treligion="'+prov[ids]['religion']+'"\n')
            file.write('\t\ttrade_goods="'+prov[ids]['trade_goods']+'"\n')
            file.write('\t\tcivilization_value='+str(prov[ids]['civilization_value'])+'\n')
            file.write('\t\tbarbarian_power='+str(prov[ids]['barbarian_power'])+'\n')
            file.write('\t\tprovince_rank="'+prov[ids]['province_rank']+'"\n')
            file.write(prov[ids]['pops']+'\n')
            file.write(prov[ids]['holysite']+'\n')
            file.write(prov[ids]['port_building']+'\n')
            file.write(prov[ids]['minor_pops1']+'\n')
            file.write(prov[ids]['minor_pops2']+'\n')
            file.write('\t}\n\n')

In [9]:
### Write the province setup as multiple files (I:Rome standard)
temp = {}
for ids in prov: # First the file that holds every inhabitable province
    if prov[ids]['pops'][-1] == '#': temp[ids] = prov[ids].copy() #In my case this is what I use to tell if a province is habitable
write_provinces('by_region/00_default', temp)

for r_name in regions: # Now by region
    temp = {}
    templist = []
    for a_name in regions[r_name]: templist.extend(areas[a_name])
    templist.sort()
    for ids in templist: temp[ids] = prov[ids].copy()
    write_provinces('by_region/00_'+r_name, temp)

### Write the province setup as 1 file
write_provinces('abw_provinces', prov)